In [ ]:
# default_exp synchro.io

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import numpy as np



In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_utils.ipynb.
Converted 02_processing.ipynb.
Converted 03_modelling.ipynb.
Converted 04_plotting.ipynb.


error: bad escape \i at position 150